In [ ]:
# analyze history
# get URLs with 300 words + 
# get tweets
# get model

In [ ]:
from proseflow.load import load

In [ ]:
load("data/browserhistory_012021.csv")

No Match found in type inference, returning None
None shape {}


'None found'

In [ ]:
import pandas as pd
history = pd.read_csv("data/browserhistory_012021.csv")

In [ ]:
h = history
len(h)

56711

In [ ]:
h

,order,id,date,time,title,url,visitCount,typedCount,transition
0,0,69921,1/3/2021,18:24:31,Export Chrome History - Chrome Web Store,https://chrome.google.com/webstore/detail/expo...,1,0,link
1,1,69920,1/3/2021,18:24:16,Export Chrome History - Chrome Web Store,https://chrome.google.com/webstore/detail/expo...,1,1,typed
2,2,66876,1/3/2021,18:04:33,Simulacrum | Jitsi Meet,https://meet.jit.si/simulacrum,3,2,typed
3,3,59477,1/3/2021,18:02:36,Meet - The Little Typer (5),https://meet.google.com/nhi-xdzs-gbx?authuser=0,10,0,link
4,4,69919,1/3/2021,18:02:35,Meet - nhi-xdzs-gbx,https://meet.google.com/nhi-xdzs-gbx?authuser=...,2,0,link
...,...,...,...,...,...,...,...,...,...
56706,56706,30533,10/5/2020,19:31:50,"Google Calendar - Sunday, January 3, 2021, today",https://calendar.google.com/calendar/u/0/r?tab=mc,247,1,link
56707,56707,30533,10/5/2020,19:31:49,"Google Calendar - Sunday, January 3, 2021, today",https://calendar.google.com/calendar/u/0/r?tab=mc,247,1,typed
56708,56708,28193,10/5/2020,19:31:41,Slack | Sneha (LD15) | EF Europe15,https://app.slack.com/client/T018XV4B7AB/D019Y...,11,0,link
56709,56709,38351,10/5/2020,19:31:41,"Slack | Spencer, EF Team | EF Europe15",https://app.slack.com/client/T018XV4B7AB/D01BU...,6,0,link


In [ ]:
any([True, False, False])

True

In [ ]:
stop_urls = ["chrome.google", "linkedin.", "libgen.", "google.com", "file:///", "google.com", "duckduckgo.com", "twitter.com", "jitsi.", "slack.com", "youtube.com", "reddit.com", "meet.", "/accounts", "/billing"  ]
# cleaned = [row for index, row in h.iterrows() if not any([url in row["url"] for url in stop_urls])]
# len(cleaned)

In [ ]:
import numpy as np
mask = [np.sum([u in v for u in stop_urls]) == 0 for v in h.url]

In [ ]:
len(h[mask])

27929

In [ ]:
import bs4

ModuleNotFoundError: No module named 'bs4'

In [ ]:
cleaned = h[mask]

In [ ]:
cleaned.url[1400:1500]

3995    https://roamresearch.com/#/app/markus/page/gr5...
3996         https://scifact.apps.allenai.org/leaderboard
3997    https://scifact.apps.allenai.org/search?Hypert...
3998    https://scifact.apps.allenai.org/search?Covid%...
3999    https://scifact.apps.allenai.org/search?Covid%...
                              ...                        
4164                        https://angel.co/p/felix-jahn
4168             https://www.vcguide.co/?3d70cb88_page=21
4169             https://www.vcguide.co/?3d70cb88_page=20
4170              https://www.nfx.com/team/shayma-sharif/
4171                            https://www.nfx.com/team/
Name: url, Length: 100, dtype: object

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup("https://dacapo.io/is-it-time-to-start-making-open-source-software-usable/", "html.parser")

/Users/markus/Library/Caches/pypoetry/virtualenvs/proseflow-GKtXBSGs-py3.8/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "https://dacapo.io/is-it-time-to-start-making-open-source-software-usable/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [ ]:
import requests
from toolz import thread_first, thread_last
from typing import List

STRING_FUNCS = ["capitalize", "count", "isalnum", "isalpha", "isascii", "isdecimal", "isdigit", "isidentifier", "islower", "isnumeric", "isspace", "istitle", "isupper", "lower", "lstrip", "replace", "rstrip", "split", "strip", "upper"]

for s_func in STRING_FUNCS: #PYTHON MAGIC
    exec("%s=getattr(str, s_func)" %s_func)
    
def pipe(*funcs:List[callable], thread="first"):
  thread = thread_first if thread == "first" else thread_last
  return lambda data: thread(data, *funcs)

clean_sentence = pipe(strip,
                     (replace, "\n", " ")
                     )

In [ ]:
def bs_to_paragraphs(soup, tag, tag_filter, strip=True):
  """ Extracts all the text from the given tags and returns them as a list. """
  return [paragraph.get_text(strip=strip) for paragraph in soup.findAll(tag, tag_filter)]

url_to_text = pipe(url_to_html,
                      html_to_bs,
                      (bs_replace, "br", "\n"),
                      bs_to_text
                     )

url_to_bs = pipe(url_to_html, 
                 html_to_bs)

In [ ]:
def url_to_html(url: str):
  """ Takes a url and returns a request.Response object """
  return requests.get(url, headers = {'User-Agent': 'test'})

def url_to_base_url(url):
  """ Gets the base url from url (useful for pagination). """
  return "/".join(url.split("/")[:3])

def html_to_bs(html, parser="lxml"):
  """ Takes a request.Response object and returns a Beautiful Soup object"""
  return BeautifulSoup(html.content, parser)

def bs_replace(soup, source, target):
  """ Replaces source tag with target tag in Beautiful Soup object """
  for tag in soup.findAll(source):
    tag.replace_with(target)
  return soup

def bs_to_text(soup, strip=True):
  """ Extracts all the text from a Beautiful soup object. """
  return soup.get_text(strip=strip)

def bs_to_paragraphs(soup, tag, tag_filter, strip=True):
  """ Extracts all the text from the given tags and returns them as a list. """
  return [paragraph.get_text(strip=strip) for paragraph in soup.findAll(tag, tag_filter)]

In [ ]:
text = url_to_text("https://dacapo.io/is-it-time-to-start-making-open-source-software-usable/")

In [ ]:
len(text.split(" "))

874

In [ ]:
def exception_handler(func):
    def inner_function(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except TypeError:
            print(f"{func.__name__} only takes numbers as the argument")
    return inner_function    

In [ ]:
articles = []
for url in cleaned.url[1400:1500]:
    print(url)
    try:
        t = url_to_text(url)
    except:
        pass
    
    if len(t.split(" ")) > 300:
        articles + [t]

https://roamresearch.com/#/app/markus/page/gr59Em0Ko
https://scifact.apps.allenai.org/leaderboard
https://scifact.apps.allenai.org/search?Hypertension%20is%20a%20common%20comorbidity%20seen%20in%20COVID-19%20patients.
https://scifact.apps.allenai.org/search?Covid%20causes%20problems%20with%20breath
https://scifact.apps.allenai.org/search?Covid%20causes%20cancer
https://scifact.apps.allenai.org/search?Covid%20is%20caused%20by%20AIDS
https://scifact.apps.allenai.org/
https://www.vcguide.co/emerging-funds
https://www.buildingsomethingnew.co/?32c5dca6_page=3
https://1lib.nl/book/1247175/9f426f
https://www.buildingsomethingnew.co/?32c5dca6_page=2
file:///Users/markus/Projects/hypermaze/Eva-Maria%20Engelen,%20Christian%20Fleischhack,%20C.%20Giovanni%20Galizia%20(auth.),%20Prof.%20Dr.%20Eva-Maria%20Engelen,%20Prof.%20Dr.%20Christian%20Fleischhack,%20Prof.%20Dr.%20C.%20Giovanni%20Galizia,%20Prof.%20Dr.%20Katharina%20Landfester%20(eds.)%20-%20Heureka_%20Evidenzkriteri%20-%20libgen.lc.pdf
http:/

In [ ]:
texts = [url_to_text(url) for url in cleaned.url[1400:1500]]

InvalidSchema: No connection adapters were found for 'file:///Users/markus/Projects/hypermaze/Eva-Maria%20Engelen,%20Christian%20Fleischhack,%20C.%20Giovanni%20Galizia%20(auth.),%20Prof.%20Dr.%20Eva-Maria%20Engelen,%20Prof.%20Dr.%20Christian%20Fleischhack,%20Prof.%20Dr.%20C.%20Giovanni%20Galizia,%20Prof.%20Dr.%20Katharina%20Landfester%20(eds.)%20-%20Heureka_%20Evidenzkriteri%20-%20libgen.lc.pdf'